In [1]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import copy

In [2]:
np.random.seed(1)
torch.manual_seed(1)
torch.set_default_dtype(torch.float)

In [3]:
device = torch.device('mps' if torch.backends.mps.is_available() else 'mps')


In [4]:
data       = np.load("./data/ecg_data.npz")
Xtr        = torch.tensor(data["Xtr"]).type(torch.float).to(device)    #Clean train
Xtr_noise  = torch.tensor(data["Xtr_noise"]).type(torch.float).to(device)  #Noisy train
Xte_noise  = torch.tensor(data["Xte_noise"]).type(torch.float).to(device)  #Noisy test

params = np.load("./data/ecg_params.npz")
W = torch.FloatTensor(params["W"]).type(torch.float).to(device)  #Decoder parameters
V = torch.FloatTensor(params["V"]).type(torch.float).to(device)  #Encoder parameters

In [5]:
class ECGDataset(Dataset):
    def __init__(self, X,R):
        self.X = X
        self.R= R

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        r = self.R[idx]
        return x,r

In [9]:
class linear_AE(nn.Module):
    def __repr__(self):
        return {"model":"LinearAE","K":self.K,"lr":self.lr,"epoch":self.epochs}
    
    def __init__(self, dim,K,epoch=1000,lr=0.001):
        super(linear_AE, self).__init__()
        self.model=nn.Sequential(
            nn.Linear(dim, K),
            nn.Linear(K,dim)
            )
        self.lr=lr
        self.epochs=epoch
        self.criterion=nn.MSELoss(reduction="mean")
        self.K=K

    def forward(self, x):
        x = self.model(x)
        return x
    
    def reconstruct(self,x,W,V):
        self.model[0].weight.data=W
        self.model[1].weight.data=V
        x=self.model(x)
        return x
    
    def loss(self,X,X_hat):
        return self.criterion(X,X_hat).item()
    
    def dataloader(self,X,R,batch_size=128):
        dataset = ECGDataset(X,R)
        dataloader=DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return dataloader
    
    def fit(self,X,R):
        train_loss_list=[]
        optimizer=optim.Adam(self.parameters(),lr=self.lr)
        for epoch in range(self.epochs+1):
            train_loss = 0.0
            self.model.train()
            optimizer.zero_grad()
            output = self.model(X)
            loss = self.criterion(output, R)
            loss.backward()
            optimizer.step()
            train_loss_list.append(loss)
            #if(epoch%100==0):print('Epoch [{}/{}], Train Loss: {:.9f}'.format(epoch, self.epochs, loss))
        #self.saver(self.model.state_dict())
        return train_loss

    def test(self,X,R):
        self.model.eval()
        with torch.no_grad():
            output=self.model(X)
            loss=self.criterion(output,R)
            return loss

    def print_param(self):
        for param in self.parameters():
            print(f'parameter shape:{param.shape}')
            #print(f'parameter value:{param.data}')
    
    @staticmethod
    def plot_loss(X,X_hat):
        time=len(X)
        plt.plot(range(1, time+1), X, label='Noisy Data')
        plt.plot(range(1, time+1), X_hat, label='Reconstructed Data')
        plt.xlabel('Time')
        plt.ylabel('ECG')
        plt.title('Noisy Data vs. Reconstructed Data')
        plt.legend()
        plt.show()

    def saver(self,best_weights,PATH="./models"):
        name=''.join(f"{key}{val}" for key, val in self.__repr__().items())
        path=PATH+"/"+name
        torch.save(best_weights,path)

In [354]:
model=linear_AE(100,5,lr=0.0001).to(device)
X_hat=model.reconstruct(Xtr_noise[:5],V,W)
bloss=model.loss(Xtr_noise[:5],X_hat)

In [355]:
model.__repr__()

{'model': 'LinearAE', 'K': 5, 'lr': 0.0001, 'epoch': 1000}

In [356]:
print(bloss)

0.646934449672699


In [361]:
model=linear_AE(100,10,lr=0.01,epoch=1500).to(device)
loss=model.fit(Xtr,Xtr)

Epoch [0/1500], Train Loss: 2.852277517
Epoch [100/1500], Train Loss: 0.206439704
Epoch [200/1500], Train Loss: 0.139985576
Epoch [300/1500], Train Loss: 0.134292096
Epoch [400/1500], Train Loss: 0.132769600
Epoch [500/1500], Train Loss: 0.132250130
Epoch [600/1500], Train Loss: 0.131983653
Epoch [700/1500], Train Loss: 0.132048294
Epoch [800/1500], Train Loss: 0.131748363
Epoch [900/1500], Train Loss: 0.131882519
Epoch [1000/1500], Train Loss: 0.131742924
Epoch [1100/1500], Train Loss: 0.131601170
Epoch [1200/1500], Train Loss: 0.132043496
Epoch [1300/1500], Train Loss: 0.131595954
Epoch [1400/1500], Train Loss: 0.132189572
Epoch [1500/1500], Train Loss: 0.131549552


In [367]:
model=linear_AE(100,10,lr=0.001,epoch=1000).to(device)
loss=model.fit(Xtr_noise,Xtr)

Epoch [0/1000], Train Loss: 2.645266294
Epoch [100/1000], Train Loss: 0.792887092
Epoch [200/1000], Train Loss: 0.450813681
Epoch [300/1000], Train Loss: 0.342895806
Epoch [400/1000], Train Loss: 0.282044500
Epoch [500/1000], Train Loss: 0.241769731
Epoch [600/1000], Train Loss: 0.214009836
Epoch [700/1000], Train Loss: 0.195932508
Epoch [800/1000], Train Loss: 0.183821037
Epoch [900/1000], Train Loss: 0.175416544
Epoch [1000/1000], Train Loss: 0.169668272


In [8]:
K=[k for k in range(5,150,5)]
training_loss_list=[]
test_loss_list=[]
for k in K:
    X_train, X_val, R_train, R_val = train_test_split(Xtr_noise, Xtr, test_size=0.2)
    model=linear_AE(100,k,lr=0.001,epoch=1000).to(device)
    train_loss=model.fit(X_train,R_train)
    test_loss=model.test(X_val,R_val)
    print(f'K:{k} Train Loss:{train_loss}   Test Loss:{test_loss}')

Test Loss:0.4173484742641449
K:5 Train Loss:0.0   Test Loss:0.4173484742641449
Test Loss:0.18094928562641144
K:10 Train Loss:0.0   Test Loss:0.18094928562641144
Test Loss:0.11624838411808014
K:15 Train Loss:0.0   Test Loss:0.11624838411808014


KeyboardInterrupt: 